# Typehints with semantic menaings:

I desire typehints that specify semantic meanings in addition to their type, like for Dict and Tuples.

I currently have a python dictionary named `sessions` of type `Dict[str, Tuple[Path, datetime]]`, an example entry would be: 
```python 
sessions['kdiba_gor01_one_2006-6-08_14-26-15']
>> {'ripple_time_bin_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_time_bin_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45)),
 'ripple_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45)),
 'laps_time_bin_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_time_bin_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45)),
 'laps_marginals_df': (Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-24_0745AM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_marginals_df).csv'),
  datetime.datetime(2024, 1, 24, 7, 45))}
```
I'd like to be able to specify the semantic menaing of the dictionary elements in the type definition, for reference later (documentation) and potentially (down the road) to enforce consistency by implementing checks on certain properties. I'm hoping for something like: `Dict[session_id:str, Tuple[file_path:Path, parsed_datetime:datetime]]` instead of `Dict[str, Tuple[Path, datetime]]`.

Dict[str, Tuple[Path, datetime]]
Dict[session_id:str, Tuple[file_path:Path, parsed_datetime:datetime]]



## Option 1: `typing.NewType` to define aliases

In [1]:
from typing import Dict, Tuple, NewType
from datetime import datetime
from pathlib import Path

# defining type aliases
session_id = NewType('session_id', str)
file_path = NewType('file_path', Path)
parsed_datetime = NewType('parsed_datetime', datetime)

# now you can define your dictionary in terms of these new types
sessions: Dict[session_id, Tuple[file_path, parsed_datetime]]

# Option 2: Inline `attrs` classes

In [ ]:
import attrs
from collections import namedtuple
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import Zscorer
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import LongShortStatsTuple

# dict(zip(['long_stats_z_scorer', 'short_stats_z_scorer', 'long_short_z_diff', 'long_short_naive_z_diff', 'is_forward_replay'], [Zscorer, Zscorer, float, float, bool]))
LongShortStatsOutput = attrs.make_class('LongShortStatsOutput', ['long_stats_z_scorer', 'short_stats_z_scorer', 'long_short_z_diff', 'long_short_naive_z_diff', 'is_forward_replay'], slots=False, order=True)  # , bases=[LongShortStatsTuple]

# attrs.define(


In [ ]:
# Creating class using make_class
SessionEntry = attrs.make_class("SessionEntry", ["file_path", "parsed_datetime"], frozen=True, slots=True)

# Assign types to attributes
attrs.set_run_validators(SessionEntry, {
    "file_path": attrs.validators.instance_of(Path),
    "parsed_datetime": attrs.validators.instance_of(datetime)
})

entry = SessionEntry(file_path=Path('/path/to/file'), parsed_datetime=datetime.now())

